这次完成一个完整的conv2d

In [1]:
import numpy as np
import torch
import torchvision

In [2]:
def corr2d(X, K):
    
    (x_0, x_1) = X.shape
    (k_0, k_1) = K.shape
    CONV = torch.zeros((x_0 - k_0 + 1, x_1 - k_1 + 1))
    
    for i in range(x_0 - k_0 + 1):
        for j in range(x_1 - k_1 + 1):
            CONV[i, j] = (X[i: i + K.shape[0], j: j + K.shape[1]] * K).sum()

    return CONV

In [3]:
X = torch.Tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.Tensor([[0, 1], [2, 3]])
assert((corr2d(X, K) == torch.Tensor([[19, 25], [37, 43]])).any().item())

In [4]:
def corr2d_in(X, K):
    
    rst = sum([corr2d(xx, K) for xx in X])
    
    return rst

def corr2d_in_out(X, K):
    
    rst = torch.stack(tuple(corr2d_in(X, k) for k in K),dim=0)
    
    return rst

def corr2d_in_out_batch(X, K):
    
    rst = torch.stack(tuple(corr2d_in_out(x, K) for x in X),dim=0)
    
    return rst

In [5]:
X = torch.randn((2, 5, 5))
K = torch.ones((5, 2, 2))
X, K

(tensor([[[ 1.0700,  0.6867, -0.9312,  0.3745, -1.8892],
          [ 1.5544, -2.1964,  0.8828, -0.6594, -0.8191],
          [ 0.2050, -1.1438, -0.4831, -0.2604, -0.4542],
          [ 0.7127,  0.2046,  0.7206, -1.0156, -0.0444],
          [ 0.4645,  0.5553, -0.3723, -1.4740,  0.0817]],
 
         [[-1.3008,  0.6960,  0.3202, -0.5770, -0.1496],
          [-1.3744, -2.6236, -0.5954, -0.2138,  0.3922],
          [-1.4007,  1.4935, -1.1257,  0.6269,  0.2618],
          [ 1.7238,  0.9078, -0.7188, -0.6049, -0.5696],
          [ 1.5764, -1.6193, -0.0395, -1.1821,  0.0884]]]),
 tensor([[[1., 1.],
          [1., 1.]],
 
         [[1., 1.],
          [1., 1.]],
 
         [[1., 1.],
          [1., 1.]],
 
         [[1., 1.],
          [1., 1.]],
 
         [[1., 1.],
          [1., 1.]]]))

In [6]:
corr2d_in_out(X, K)

tensor([[[-3.4881, -3.7609, -1.3993, -3.5413],
         [-5.4859, -5.7916, -1.8280, -1.1259],
         [ 2.7031, -0.1448, -2.8610, -2.0605],
         [ 4.5259, -0.3616, -4.6866, -4.7206]],

        [[-3.4881, -3.7609, -1.3993, -3.5413],
         [-5.4859, -5.7916, -1.8280, -1.1259],
         [ 2.7031, -0.1448, -2.8610, -2.0605],
         [ 4.5259, -0.3616, -4.6866, -4.7206]],

        [[-3.4881, -3.7609, -1.3993, -3.5413],
         [-5.4859, -5.7916, -1.8280, -1.1259],
         [ 2.7031, -0.1448, -2.8610, -2.0605],
         [ 4.5259, -0.3616, -4.6866, -4.7206]],

        [[-3.4881, -3.7609, -1.3993, -3.5413],
         [-5.4859, -5.7916, -1.8280, -1.1259],
         [ 2.7031, -0.1448, -2.8610, -2.0605],
         [ 4.5259, -0.3616, -4.6866, -4.7206]],

        [[-3.4881, -3.7609, -1.3993, -3.5413],
         [-5.4859, -5.7916, -1.8280, -1.1259],
         [ 2.7031, -0.1448, -2.8610, -2.0605],
         [ 4.5259, -0.3616, -4.6866, -4.7206]]])

In [7]:
class Conv2D_layer(torch.nn.Module):
    
    def __init__(self, kernel_size, in_channels, out_channels):
        
        super().__init__()
        
        if isinstance(kernel_size, tuple):
            
            self.weight = torch.nn.Parameter(torch.randn(out_channels, kernel_size[0], kernel_size[1]))
        
        else:
            
            self.weight = torch.nn.Parameter(torch.randn(out_channels, kernel_size, kernel_size))
            
        
        self.bias = torch.nn.Parameter(torch.zeros(out_channels, 1, 1))
        
        self.in_channel = in_channels
        
    def forward(self, X):
        
        # if len(X.shape) == 2:
        #     X = X.reshape([1, self.in_channel] + X.shape)
        # elif len(X.shape) == 3:
        #     X = X.reshape([1, self.in_channel] + list(X.shape)[1:])
        # else:
        #     shape_list = list(X.shape)
        #     shape_list[1] = self.in_channel
        #     X = X.reshape(shape_list)
                        
        return corr2d_in_out_batch(X, self.weight) + self.bias
